<a href="https://colab.research.google.com/github/Taichi-Fujiwara/SIGNATE_opt_recommendation_20220102/blob/main/SIGNATE_OPT_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# 数字把握

In [ ]:
# !pip install --upgrade pandas_profiling

In [ ]:
#ライブラリ
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
# from xgboost import XGBRFClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix

from pylab import rcParams

In [ ]:
# csvデータ取得
# trainPath = "/content/drive/MyDrive/Colab Notebooks/SIGNATE/20220102_opt/train"
# testPath = "/content/drive/MyDrive/Colab Notebooks/SIGNATE/20220102_opt/test.tsv"
# sampleSubmitPath = "/content/drive/MyDrive/Colab Notebooks/SIGNATE/20220102_opt/sample_submit.tsv"
trainPath = "train"
testPath = "test.tsv"
sampleSubmitPath = "sample_submit.tsv"

# 目的変数
target= "product_id"
nDCG = 0

trainA = pd.read_table(trainPath+"/train_A.tsv")
trainB = pd.read_table(trainPath+"/train_B.tsv")
trainC = pd.read_table(trainPath+"/train_C.tsv")
trainD = pd.read_table(trainPath+"/train_D.tsv")

test = pd.read_table(testPath)

# 応募用サンプルファイル
sample_submit = pd.read_table(sampleSubmitPath)

# print(
#     train.shape,
#     test.shape,
#     sample_submit.shape
# )

event_type* event_type：行動種別（コンバージョン:3, クリック:2, 閲覧:1, カート:0） 

* ad：(cv)コンバージョンが広告経由か否か。コンバージョンではない場合は-1

* train_A.tsv, train_B.tsv, train_C.tsv, train_D.tsvはそれぞれ人材、旅行、不動産、アパレルのデータです

* time_stamp：yyyy-mm-dd hh:mm:ss.sss

In [ ]:
# 人材
trainA.value_counts(subset=['event_type'])

event_type
1             1981780
0             1286738
3               74561
2               33414
dtype: int64

In [ ]:
# 旅行
trainB.value_counts(subset=['event_type'])

event_type
1             1035637
0               52639
3               21710
2                3439
dtype: int64

In [ ]:
# 不動産
trainC.value_counts(subset=['event_type'])

event_type
1             1650622
2               14670
3                1358
dtype: int64

In [ ]:
# アパレル
trainD.value_counts(subset=['event_type'])

event_type
0             2522371
1             1049690
3                 781
dtype: int64

In [ ]:
trainA.value_counts(subset=['product_id'])

product_id
00005175_a    11688
00013874_a     9742
00004347_a     9193
00001225_a     8368
00005844_a     8172
              ...  
00010035_a        1
00012151_a        1
00005353_a        1
00003302_a        1
00007373_a        1
Length: 13866, dtype: int64

---
# 特徴量作成

In [ ]:
# 時間を分ける
# time_stamp：yyyy-mm-dd hh:mm:ss.sss

trainA["year"] = trainA["time_stamp"].str[:4].astype('int')
trainA["month"] = trainA["time_stamp"].str[5:7].astype('int')
trainA["day"] = trainA["time_stamp"].str[8:10].astype('int')
trainA["hour"] = trainA["time_stamp"].str[11:13].astype('int')
trainA["min"] = trainA["time_stamp"].str[14:16].astype('int')
trainA["sec"] = trainA["time_stamp"].str[17:19].astype('int')

trainB["year"] = trainB["time_stamp"].str[:4].astype('int')
trainB["month"] = trainB["time_stamp"].str[5:7].astype('int')
trainB["day"] = trainB["time_stamp"].str[8:10].astype('int')
trainB["hour"] = trainB["time_stamp"].str[11:13].astype('int')
trainB["min"] = trainB["time_stamp"].str[14:16].astype('int')
trainB["sec"] = trainB["time_stamp"].str[17:19].astype('int')

trainC["year"] = trainC["time_stamp"].str[:4].astype('int')
trainC["month"] = trainC["time_stamp"].str[5:7].astype('int')
trainC["day"] = trainC["time_stamp"].str[8:10].astype('int')
trainC["hour"] = trainC["time_stamp"].str[11:13].astype('int')
trainC["min"] = trainC["time_stamp"].str[14:16].astype('int')
trainC["sec"] = trainC["time_stamp"].str[17:19].astype('int')

trainD["year"] = trainD["time_stamp"].str[:4].astype('int')
trainD["month"] = trainD["time_stamp"].str[5:7].astype('int')
trainD["day"] = trainD["time_stamp"].str[8:10].astype('int')
trainD["hour"] = trainD["time_stamp"].str[11:13].astype('int')
trainD["min"] = trainD["time_stamp"].str[14:16].astype('int')
trainD["sec"] = trainD["time_stamp"].str[17:19].astype('int')

---
モデリング

In [ ]:
# xgboostで実装

#データ結合
train_all = pd.concat([trainA,trainB,trainC,trainD])
train_y_all = pd.DataFrame(
    data={
        'product_id': train_all[target]
         })

length = train_all.shape[0]

df_all = pd.concat([train_all,test])

print("trainA :",trainA.shape)
print("df_all :",df_all.shape)

# df_all.drop(target, axis = 1)

# ダミー変数化
# df_all = pd.get_dummies(df_all)

trainA : (3376493, 11)
df_all : (9741008, 11)


In [ ]:
# カテゴリ変数のラベリング
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# 目的変数の対応表作成
LE = LabelEncoder()
df_all_targegt_index = pd.DataFrame(
    data={
        'product_id': df_all["product_id"],
        'product_index':LE.fit_transform(df_all["product_id"].values)
         })

train_y_all_index = pd.DataFrame(
    data={
        'product_id': train_y_all["product_id"],
        'product_index':LE.fit_transform(train_y_all["product_id"].values)
         })


# ラベルエンコーディング
df_all["product_id"] = LE.fit_transform(df_all["product_id"].values)
df_all["user_id"] = LE.fit_transform(df_all["user_id"].values)
df_all["time_stamp"] = LE.fit_transform(df_all["time_stamp"].values)
train_y_all["product_id"] = LE.fit_transform(train_y_all["product_id"].values)

In [ ]:
#df_allを訓練データとテストデータに再度分ける
new_train = df_all.iloc[:length,:]
new_test = df_all.iloc[length:,:]

new_train

,user_id,product_id,event_type,ad,time_stamp,year,month,day,hour,min,sec
0,0,36766,1.0,-1.0,2166144,2017.0,4.0,8.0,12.0,9.0,4.0
1,0,36766,1.0,-1.0,7574540,2017.0,4.0,27.0,12.0,55.0,57.0
2,0,55915,1.0,-1.0,2163119,2017.0,4.0,8.0,11.0,57.0,53.0
3,0,4989,1.0,-1.0,2164893,2017.0,4.0,8.0,12.0,4.0,26.0
4,0,13180,1.0,-1.0,2165187,2017.0,4.0,8.0,12.0,5.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...
3572837,231809,359064,0.0,-1.0,6801738,2017.0,4.0,25.0,0.0,30.0,27.0
3572838,231809,364829,0.0,-1.0,6809150,2017.0,4.0,25.0,1.0,0.0,55.0
3572839,231809,116774,0.0,-1.0,6801472,2017.0,4.0,25.0,0.0,29.0,19.0
3572840,231809,333121,0.0,-1.0,6801300,2017.0,4.0,25.0,0.0,28.0,37.0


In [ ]:
# Cross Validation用にデータ分ける
seed = 0;
X_train, X_test, y_train, y_test = train_test_split(new_train, train_y_all, 
                                                    test_size = .3,
                                                    random_state = seed,
                                                    # stratify = train_y_all
                                                    )

In [ ]:
param = {
    'max_depth': 6, 
    'eta': 0.1, 
    # 'objective': 'multi:softmax',
    # 'num_class': len(df_all["product_id"].unique()),
    'objective':'reg:linear',
    'eval_metric':'rmse'
}
# objective = 
# https://qiita.com/FJyusk56/items/0649f4362587261bd57a

# XGBoost のモデリング
# xgb = XGBRFClassifier()
dm_train = xgb.DMatrix(X_train, label=y_train)
# xgb.fit(X_train, y_train)
model = xgb.train(param, dm_train, num_boost_round=1000)
# xgb.score(X_test, y_test)

ax = xgb.plot_importance(model)
fig = ax.figure
fig.set_size_inches(10, 13)

[13:16:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
